In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import tensorflow as tf
import pickle
import math, sys, getopt

modelout = 'model'
imagewidth = 1024
imageheight = 680

print ('Output file is "', modelout)
print ('Input image size is "', imagewidth, ", ", imageheight)

if(modelout == '' or imagewidth == 0 or imageheight == 0):
    print('Missing required parameter.')
    print ('train.py -o <modelout> -w <inputwidth> -h <inputheight>')
    sys.exit(2)

print ('--------------------\n\n')

K.image_data_format()

img_width, img_height = imagewidth, imageheight

initializer = tf.keras.initializers.GlorotNormal()

model = models.Sequential()
model.add(layers.Conv2D(32, (132, 88),input_shape=(img_width, img_height, 3), strides=(3,2), activation="relu", kernel_initializer="he_uniform"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (66, 66), strides=(2,2), activation="relu",kernel_initializer="he_uniform"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (16, 16), strides=(2,2), activation="relu",kernel_initializer="he_uniform"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(256, (7, 7), strides=(2,2), activation="relu",kernel_initializer="he_uniform"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(512, (3, 3), strides=(1,1), activation="relu",kernel_initializer="he_uniform"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation="relu",kernel_initializer="he_uniform"))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(128, activation="relu",kernel_initializer="he_uniform"))
model.add(layers.Dropout(0.2))

model.add(Dense(1, input_dim=1, kernel_initializer="he_uniform", activation="linear"))

model.compile(
    loss='mse',
    optimizer=optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0))

model.summary() 

run_filepath = modelout
model.save(run_filepath)



Output file is " model
Input image size is " 1024 ,  680
--------------------


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 297, 32)      1115168   
_________________________________________________________________
batch_normalization_5 (Batch (None, 298, 297, 32)      128       
_________________________________________________________________
dropout_7 (Dropout)          (None, 298, 297, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 117, 116, 64)      8921152   
_________________________________________________________________
batch_normalization_6 (Batch (None, 117, 116, 64)      256       
_________________________________________________________________
dropout_8 (Dropout)          (None, 117, 116, 64)      0         
________________________________________

OSError: Unable to create file (unable to open file: name = 'model', errno = 21, error message = 'Is a directory', flags = 13, o_flags = 242)

In [4]:
#!/usr/bin/env python

import skimage
import os
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
from skimage import io
from skimage.transform import resize
import sys, getopt
from tensorflow.keras import initializers, models, optimizers
import os
import tensorflow as tf
from tensorflow.python.platform.tf_logging import error

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self,
                 batch_size,path,
                 input_size=(1024, 680, 3),
                 shuffle=True):
        
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.path = path
        self.items = glob(os.path.join(self.path,"*.jpg"))
        self.n = len(self.items)
        
    
    def on_epoch_end(self):
        pass
    
    def __getitem__(self, index):
        #import random

        x = [] # images as arrays
        y = [] # labels Infiltration or Not_infiltration
        WIDTH = 1024
        HEIGHT = 680
        j = 0
        images = []
        rawscore = 0.0

        for i in range(index*self.batch_size, (index+1)*self.batch_size):
            item = self.items[index]
            #print("Reading " + item)
            # Read and resize image
            full_size_image = io.imread(item)
            rawscore = int(os.path.basename(item).split("-")[0])
            out = rawscore
            y.append(out)
            images.append(item)
            resizedImage = resize(full_size_image, (WIDTH,HEIGHT), anti_aliasing=True) 
            if(len(resizedImage.shape) < 3):
                resizedImage = skimage.color.gray2rgb(resizedImage)

            x.append(resizedImage)
        return np.array(x),np.array(y)
    
    def __len__(self):
        return self.n // self.batch_size

class WaitCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            import time

            stream = os.popen("nvidia-smi -q -i 0 -d TEMPERATURE|grep 'GPU Current'|cut -d\" \" -f 30")
            temp = int(stream.read())
            print("The temperature of the GPU is ",temp)

            if(temp>75):
                print("Cooling down...")
                while(temp>55):
                    time.sleep(15)
                    stream = os.popen("nvidia-smi -q -i 0 -d TEMPERATURE|grep 'GPU Current'|cut -d\" \" -f 30")
                    temp = int(stream.read())

                print("Resuming -> ")

            return super().on_epoch_end(epoch, logs=logs)

def proc_image_dir(Images_Path):
    
#    image_classes = sorted([dirname for dirname in os.listdir(Images_Path)
#                      if os.path.isdir(os.path.join(Images_Path, dirname)) and not dirname.startswith(".") and not dirname.startswith("mblur")])
    
#    print(image_classes)
    
    x = [] # images as arrays
    y = [] # labels Infiltration or Not_infiltration
    WIDTH = 1024
    HEIGHT = 680
  
    print("Adding Images: ",end="")
    i = 0
#    for image_class in image_classes:
    #print("Processing ", image_class)
    items = glob(os.path.join(Images_Path,"*"))
    j = 0
    images = []
    rawscore = 0.0

    for item in items:
        print("Reading "+item)
        if item.lower().endswith(".jpg") or item.lower().endswith(".bmp"):
            # Read and resize image
            full_size_image = io.imread(item)

            rawscore = int(os.path.basename(item).split("-")[0])

            if rawscore >= 1:
                #x.append(full_size_image)
                out = rawscore
                print(out)
                y.append(out)
                images.append(item)
                resizedImage = resize(full_size_image, (WIDTH,HEIGHT), anti_aliasing=True) 
                if(len(resizedImage.shape) < 3):
                    resizedImage = skimage.color.gray2rgb(resizedImage)

                x.append(resizedImage)
                j+=1
            #if j>3:
            #    break
                

    print("\nRead " + str(j) + " images.\n\n")
    return x,y,images

def init_layer(layer):
    try:
        initializer = tf.keras.initializers.GlorotUniform()
        import keras.backend as K
        session = K.get_session()
        if hasattr(layer, 'kernel_initializer'):
            print("initializing kernel weights")
            layer.kernel.initializer.run(session=session)
        if hasattr(layer, 'bias_initializer'):
            print("initializing bias weights")
            layer.bias.initializer.run(session=session) 
        print(layer.name," re-initilized")
    except:
        print(layer.name, " could not be re-initilized", sys.exc_info())

def train(modelin,modelout,imagepath,epochs,batch_size,lr,decay,nesterov,checkpoint_filepath,train_path,val_path,transfer_learning,randomize_weights):
    #load model
    model = models.load_model(modelin)

    if transfer_learning:
        for layer in model.layers:
            if (isinstance(layer, tf.keras.layers.Conv2D)):
                print("conv layer",layer.name)
                layer.trainable = False

    if randomize_weights:
        
        for layer in model.layers:
            if(layer.trainable):
                init_layer(layer)

    model.build()

    print(model.summary())

    if(imagepath!=''):
        #load images
        x2,y2,images = proc_image_dir(imagepath)
        
        # First split the data in two sets, 60% for training, 40% for Val/Test)
        X_train, X_valtest, y_train, y_valtest = train_test_split(x2,y2, test_size=0.4, random_state=1)

        # Second split the 40% into validation and test sets
        X_test, X_val, y_test, y_val = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=1)
    else:
        training_generator = CustomDataGen(batch_size,train_path)
        validation_generator = CustomDataGen(batch_size,val_path)
        #X_train,y_train,image_list_train = proc_image_dir(train_path)
        #X_val,y_val,image_list_val = proc_image_dir(val_path)

    #run training loop
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,   
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    wait_callback = WaitCallback()

    model.compile(
        loss='mae',
        optimizer=optimizers.SGD(learning_rate=lr,momentum = 0.0, decay=decay, nesterov=nesterov))
    #history = model.fit(np.array(X_train), np.array(y_train),
    #    validation_data=(np.array(X_val), np.array(y_val)),
    #        epochs=epochs, batch_size=batch_size,
    #        callbacks=[model_checkpoint_callback,wait_callback])

    history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=epochs,callbacks=[model_checkpoint_callback,wait_callback],
                    use_multiprocessing=True,
                    workers=6)

    #save model
    model.save(modelout)

    print ([model.history.history["loss"],model.history.history["val_loss"]])

modelin = 'model'
modelout = 'model.trained'
imagepath = ''
epochs = 10
batch_size = 10
lr = .001
decay = 0.0001
nesterov = False
checkpoint_filepath = "./checkpoint/"
train_path = './databaserelease2/NatureDataset/train'
val_path = './databaserelease2/NatureDataset/val'
transfer_learning = False
randomize_weights = False

print ('Input file is "', modelin)
print ('Output file is "', modelout)
print ('Image path is "', imagepath)

if(modelin == '' or modelout == '' or (imagepath == '' and (train_path == '' or val_path == ''))):
    print('Missing required parameter.')
    print ('train.py -i <modelin> -o <modelout> -p <imagepath>')
    sys.exit(2)


print ('--------------------\n\n')

train(modelin,modelout,imagepath,epochs,batch_size,lr,decay,nesterov,checkpoint_filepath,train_path,val_path,transfer_learning,randomize_weights)




ModuleNotFoundError: No module named 'skimage'